In [1]:
#Block
blockchain = [hash('Genesis')]
blocks = 0
block= ["genesis"]

#EXECUTING THIS RESETS THE CHAIN

def transact(origin_key,destiny_key,content_sn):
    
    if content_sn in origin_key:
    
        previous_hash = blockchain[-1]
        transaction_hash = hash("{}{}{}".format(origin_key,destiny_key,content_sn))
        block_hash = hash('{}{}'.format(previous_hash,transaction_hash))
        print('Part sent!')
        blockchain.append(block_hash)
        
        #transfering the part
        
        origin_key.pop(origin_key.index(content_sn))
        destiny_key.append(content_sn)
        

    #hash count
    
        global blocks
        blocks = blocks + 1
    
    #creating blocks database
    
        block.append([origin_key,destiny_key,content_sn])
    
    else:
        
        print("content not in your ledger!")
        
     #block explorer
    
def blockData(block):
    
    print ('key origem:{}'.format(block[0][0]))
    print ('key destino:{}'.format(block[1][0]))
    print ('serial number:{}'.format(block[2]))
    
    #Wallet

def wallet(key):
    for item in key:
        print(item)

    #open wallet
    
def generate_wallet():
    import string
    import random
    print('Your wallet is: {}'.format(''.join(random.choices(string.ascii_uppercase + string.digits, k=8))))
    print('Keep it safe!')

#criar def Register(key)
    

In [122]:
from hashlib import sha256
import json
import time

class blockchain:
    def __init__(self, chain):
        self.chain = []
    
    def genesis(self):
        genesis = block(0,[],time.time(),'0')
        genesis.hash = genesis.compute_hash()
        self.chain.append(genesis)

#block

class block:
    def __init__(self, index, transaction, timestample, prev_hash):
        self.index = index
        self.transaction = transaction
        self.timestample = timestample
        self.prev_hash = prev_hash
        
    def compute_hash(self):
    
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()
    
class transaction:
    def __init__(self, SN, from_key, to_key, validation):
        self.SN = SN
        self.from_key = from_key
        self.to_key = to_key
        self.validation = validation

def send(from_key, to_key, SN):
    if SN in from_key.parts:
        from_key.parts.pop(from_key.parts.index(SN))
        to_key.parts.append(SN)
        print('part sent')
    
    elif SN not in from_key.parts:
        print('Part not in your wallet')
    
    
class wallet:
    def __init__(self,parts,public_key,private_key):
        self.parts = parts
        self.public_key = public_key
        self.private_key = private_key

    import random
    import string
    def new_wallet(username):    #não funciona 
        username = wallet([],(''.join(random.choices(string.ascii_uppercase + string.digits, k=8))),123)
        


In [123]:
blockchain.genesis()

TypeError: genesis() missing 1 required positional argument: 'self'

In [107]:
genesis

NameError: name 'genesis' is not defined

In [69]:
a = wallet([123],'a','a1')
b = wallet([],'b','b1')

In [67]:
send(a,b,123)

part sent
